In [1]:
import pandas as pd
import numpy as np
import os

# Konfiguracja i wczytanie danych
file_X = 'War2_X.csv'
file_Xprime = 'War2_Xprime.csv'

# Wczytanie z uwzględnieniem średnika jako separatora i przecinka jako separatora dziesiętnego
df_X = pd.read_csv(file_X, sep=';', decimal=',', header=None)
df_Xprime = pd.read_csv(file_Xprime, sep=';', decimal=',', header=None)

# Przygotowanie danych
X_raw = df_X.iloc[:, 1:].values
Xprime_raw = df_Xprime.iloc[:, 1:].values

X = X_raw.T
Xprime = Xprime_raw.T

print(f"Wymiary macierzy X: {X.shape}")
print(f"Wymiary macierzy X': {Xprime.shape}")

# Definicja funkcji DMD
def calculate_dmd_matrices(X, Xprime, r):
    """
    Oblicza macierze DMD dla zadanego rzędu obcięcia r.
    Zwraca Atilde (macierz w podprzestrzeni) oraz A_full (aproksymacja w pełnej przestrzeni).
    """
    # Krok 1: SVD
    U, Sigma, VT = np.linalg.svd(X, full_matrices=False)
    
    # Obcięcie do rzędu r
    Ur = U[:, :r]
    Sigmar = np.diag(Sigma[:r])
    VTr = VT[:r, :]
    
    # Krok 2: Obliczenie Atilde (zredukowana macierz dynamiki)
    M = Ur.T @ Xprime @ VTr.T
    Atilde = np.linalg.solve(Sigmar.T, M.T).T
    
    # Rekonstrukcja pełnej macierzy A (jeśli potrzebna)
    # A_approx = X' * V * inv(Sigma) * U^T
    Sigma_inv = np.diag(1.0 / Sigma[:r])
    A_full = Xprime @ VTr.T @ Sigma_inv @ Ur.T
    
    return Atilde, A_full

# Obliczenia
r = 21 

Atilde, A_full = calculate_dmd_matrices(X, Xprime, r)

print("-" * 30)
print(f"Obliczono dla rzędu r = {r}")
print(f"Wymiary macierzy Atilde: {Atilde.shape}")
print(f"Wymiary pełnej macierzy A: {A_full.shape}")
print("-" * 30)
print("Fragment macierzy Atilde (3x3):")
print(Atilde[:3, :3])

# Zapis wyników do CSV
pd.DataFrame(Atilde).to_csv('Wynik_Atilde.csv', sep=';', decimal=',', header=False, index=False)
pd.DataFrame(A_full).to_csv('Wynik_A_full.csv', sep=';', decimal=',', header=False, index=False)

print("\nZapisano wyniki do plików: 'Wynik_Atilde.csv' oraz 'Wynik_A_full.csv'")

Wymiary macierzy X: (35, 23)
Wymiary macierzy X': (35, 23)
------------------------------
Obliczono dla rzędu r = 21
Wymiary macierzy Atilde: (21, 21)
Wymiary pełnej macierzy A: (35, 35)
------------------------------
Fragment macierzy Atilde (3x3):
[[ 1.21528665e+01 -1.06699605e+02 -1.32713445e+03]
 [-2.07904233e-07 -8.82033360e+00 -1.09568331e+02]
 [-4.49142831e-08  9.00727776e-01 -1.25755996e+00]]

Zapisano wyniki do plików: 'Wynik_Atilde.csv' oraz 'Wynik_A_full.csv'
